# Chicago Crimes | Past 10 years | Cleaning pt.2 
[Information on thie dataset](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2)

In [1]:
# Import our dependencies
import requests
import time
from datetime import datetime
import pandas as pd
import numpy as np

#  Import and read the charity_data.csv.
cleaned_df_pt2 = pd.read_csv("Resources/Chicago_Crimes_cleaned_pt1.csv")
cleaned_df_pt2.head()

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,District,Community Area,Latitude,Longitude,Location
0,12283861,02/02/2021 12:57:00 AM,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,STREET,True,False,3.0,69.0,41.763110,-87.615232,"(41.763110324, -87.615231799)"
1,11825618,09/12/2019 11:30:00 AM,THEFT,RETAIL THEFT,GROCERY FOOD STORE,False,False,12.0,28.0,41.866989,-87.657205,"(41.866988841, -87.657205006)"
2,12568929,12/16/2021 03:58:00 AM,CRIMINAL DAMAGE,TO PROPERTY,RESTAURANT,False,False,6.0,44.0,41.736520,-87.613403,"(41.736520302, -87.613403026)"
3,12542403,11/14/2021 12:00:00 PM,BURGLARY,FORCIBLE ENTRY,RESIDENCE - GARAGE,False,False,12.0,24.0,41.901872,-87.699330,"(41.901872469, -87.699330295)"
4,12552301,11/24/2021 04:15:00 AM,BATTERY,SIMPLE,CTA TRAIN,False,False,16.0,76.0,41.978108,-87.904123,"(41.978108446, -87.904122758)"


In [2]:
# Get number of observations/rows in cleaned_df_pt2
len_cleaned_df_pt2 = len(cleaned_df_pt2)
numbers_cleaned_df_pt2 = "{:,}".format(len_cleaned_df_pt2)
print(numbers_cleaned_df_pt2)

50,000


In [3]:
# View colums
cleaned_df_pt2.dtypes

ID                        int64
Date                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
District                float64
Community Area          float64
Latitude                float64
Longitude               float64
Location                 object
dtype: object

### Rename columns to better fit their definition:
- ID
- Date > **DateTime**
- Primary Type > **Offense**
- Description > **Offense_Description**
- Location Description > **Place**
- Arrest
- Domestic
- District > **Police_Dist**
- Community Area > **Community_No**
- Latitude
- Longitude
- Location > **LatLong**

In [4]:
# Rename encoded columns
cleaned_df_pt2.rename(columns = {'Date':'DateTime',
                                 'Primary Type':'Offense',
                                 'Description':'Offense_Description',
                                 'Location Description':'Place',
                                 'District':'Police_Dist',
                                 'Community Area':'Community_No',
                                 'Location':'LatLong'}, 
                      inplace = True)
cleaned_df_pt2.head()

,ID,DateTime,Offense,Offense_Description,Place,Arrest,Domestic,Police_Dist,Community_No,Latitude,Longitude,LatLong
0,12283861,02/02/2021 12:57:00 AM,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,STREET,True,False,3.0,69.0,41.763110,-87.615232,"(41.763110324, -87.615231799)"
1,11825618,09/12/2019 11:30:00 AM,THEFT,RETAIL THEFT,GROCERY FOOD STORE,False,False,12.0,28.0,41.866989,-87.657205,"(41.866988841, -87.657205006)"
2,12568929,12/16/2021 03:58:00 AM,CRIMINAL DAMAGE,TO PROPERTY,RESTAURANT,False,False,6.0,44.0,41.736520,-87.613403,"(41.736520302, -87.613403026)"
3,12542403,11/14/2021 12:00:00 PM,BURGLARY,FORCIBLE ENTRY,RESIDENCE - GARAGE,False,False,12.0,24.0,41.901872,-87.699330,"(41.901872469, -87.699330295)"
4,12552301,11/24/2021 04:15:00 AM,BATTERY,SIMPLE,CTA TRAIN,False,False,16.0,76.0,41.978108,-87.904123,"(41.978108446, -87.904122758)"


## Bin columns to combine highly coorelated values and remove low variance values for modeling.

### Bin the Offenses

In [5]:
# Look at Offense value counts for binning
cleaned_df_pt2.Offense.value_counts()

THEFT                                10586
BATTERY                               9752
CRIMINAL DAMAGE                       5638
ASSAULT                               4433
DECEPTIVE PRACTICE                    3645
OTHER OFFENSE                         3303
MOTOR VEHICLE THEFT                   2240
NARCOTICS                             1978
BURGLARY                              1811
WEAPONS VIOLATION                     1810
ROBBERY                               1776
CRIMINAL TRESPASS                     1058
OFFENSE INVOLVING CHILDREN             435
PUBLIC PEACE VIOLATION                 254
CRIMINAL SEXUAL ASSAULT                252
SEX OFFENSE                            232
INTERFERENCE WITH PUBLIC OFFICER       190
HOMICIDE                               169
ARSON                                  100
CRIM SEXUAL ASSAULT                     73
PROSTITUTION                            67
STALKING                                59
CONCEALED CARRY LICENSE VIOLATION       38
LIQUOR LAW 

Decision was made to logically choose offenses because high value may not indicate a serious crime.

**Highest Priority Offenses:**
1. KIDNAPPING 28
2. HOMICIDE 104
3. ASSAULT 3662
4. BATTERY 9390
5. OFFENSE INVOLVING CHILDREN 393
6. NARCOTICS 3312
7. MOTOR VEHICLE THEFT 2197
8. BURGLARY 2459
9. THEFT 11344
10. Merge: CRIM SEXUAL ASSAULT 168 CRIMINAL SEXUAL ASSAULT 69   SEX OFFENSE 180 = 'SEXUAL CRIME'
11. CRIMINAL DAMAGE 5407
12. DECEPTIVE PRACTICE 3013

**Other Offenses:**

13. Group into VIOLENT Other:
- INTIMIDATION 27
- STALKING 47
- ROBBERY 1917
- INTERFERENCE WITH PUBLIC OFFICER 193

14. Group into NON-VIOLENT Other:
- ARSON 80
- GAMBLING 52
- LIQUOR LAW VIOLATION 48
- OBSCENITY 21
- CONCEALED CARRY LICENSE VIOLATION 12
- PUBLIC INDECENCY 5        
- NON-CRIMINAL 3
- NON - CRIMINAL 1
- OTHER NARCOTIC VIOLATION 1
- PROSTITUTION 178
- PUBLIC PEACE VIOLATION 359
- WEAPONS VIOLATION 1019
- CRIMINAL TRESPASS 1273
- OTHER OFFENSE 3038

In [6]:
merge_sex_assult_values = ['CRIM SEXUAL ASSAULT', 'CRIMINAL SEXUAL ASSAULT', 'SEX OFFENSE']

# Replace in DataFrame
for Offense in cleaned_df_pt2:
    cleaned_df_pt2.Offense = cleaned_df_pt2.Offense.replace(
        merge_sex_assult_values,"SEXUAL CRIME")


In [7]:
VIOLENT_OTHER_list = ['INTIMIDATION', 'STALKING', 'ROBBERY', 'INTERFERENCE WITH PUBLIC OFFICER']

# Replace in DataFrame
for Offense in cleaned_df_pt2:
    cleaned_df_pt2.Offense = cleaned_df_pt2.Offense.replace(
        VIOLENT_OTHER_list,"SEXUAL VIOLENT_OTHER")

In [8]:
NON_VIOLENT_OTHER_list = ['ARSON',
                          'GAMBLING',
                          'LIQUOR LAW VIOLATION',
                          'OBSCENITY',
                          'CONCEALED CARRY LICENSE VIOLATION',
                          'PUBLIC INDECENCY',
                          'NON-CRIMINAL',
                          'NON - CRIMINAL',
                          'OTHER NARCOTIC VIOLATION',
                          'PROSTITUTION',
                          'PUBLIC PEACE VIOLATION',
                          'WEAPONS VIOLATION',
                          'CRIMINAL TRESPASS',
                          'OTHER OFFENSE']

# Replace in DataFrame
for Offense in cleaned_df_pt2:
    cleaned_df_pt2.Offense = cleaned_df_pt2.Offense.replace(
        NON_VIOLENT_OTHER_list,"NON-VIOLENT OTHER")

# Check to make sure binning was successful
cleaned_df_pt2.Offense.value_counts()

THEFT                         10586
BATTERY                        9752
NON-VIOLENT OTHER              6684
CRIMINAL DAMAGE                5638
ASSAULT                        4433
DECEPTIVE PRACTICE             3645
MOTOR VEHICLE THEFT            2240
SEXUAL VIOLENT_OTHER           2048
NARCOTICS                      1978
BURGLARY                       1811
SEXUAL CRIME                    557
OFFENSE INVOLVING CHILDREN      435
HOMICIDE                        169
KIDNAPPING                       21
HUMAN TRAFFICKING                 3
Name: Offense, dtype: int64

## Bin the 'Places' column to better work for modeling
Most are of similar places, so we'll group those first.
Then we'll take the top 10 values with the most occurences, keep those, and put the rest into an 'other' bin.

In [9]:
# Check the number of unique values in each column

Place_counts = cleaned_df_pt2.Place.value_counts()
pd.options.display.max_rows = 4000
Place_counts

STREET                                                   11868
APARTMENT                                                 8662
RESIDENCE                                                 8351
SIDEWALK                                                  3266
SMALL RETAIL STORE                                        1364
ALLEY                                                     1095
RESTAURANT                                                1052
PARKING LOT / GARAGE (NON RESIDENTIAL)                     916
OTHER                                                      798
VEHICLE NON-COMMERCIAL                                     742
GAS STATION                                                723
DEPARTMENT STORE                                           717
OTHER (SPECIFY)                                            585
GROCERY FOOD STORE                                         583
PARKING LOT/GARAGE(NON.RESID.)                             566
RESIDENCE - PORCH / HALLWAY                            

In [10]:
merge_airport_values = ['AIRCRAFT',
                        'AIRPORT BUILDING NON-TERMINAL - NON-SECURE AREA',
                        'AIRPORT BUILDING NON-TERMINAL - SECURE AREA',
                        'AIRPORT EXTERIOR - NON-SECURE AREA',
                        'AIRPORT PARKING LOT',
                        'AIRPORT TERMINAL LOWER LEVEL - NON-SECURE AREA',
                        'AIRPORT TERMINAL LOWER LEVEL - SECURE AREA',
                        'AIRPORT TERMINAL UPPER LEVEL - NON-SECURE AREA',
                        'AIRPORT TERMINAL UPPER LEVEL - SECURE AREA',
                        'AIRPORT TRANSPORTATION SYSTEM (ATS)',
                        'AIRPORT VENDING ESTABLISHMENT',
                        'AIRPORT/AIRCRAFT']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Place = cleaned_df_pt2.Place.replace(
        merge_airport_values,"AIRPORT/AIRCRAFT")

In [11]:
merge_residence_values = ['APARTMENT',
                          'HOUSE',
                          'RESIDENCE',
                          'RESIDENCE - GARAGE',
                          'RESIDENCE - PORCH / HALLWAY',
                          'RESIDENCE - YARD (FRONT / BACK)',
                          'RESIDENCE PORCH/HALLWAY',
                          'RESIDENCE-GARAGE',
                          'RESIDENTIAL YARD (FRONT/BACK)',
                          'DRIVEWAY - RESIDENTIAL',
                          'YARD']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Place = cleaned_df_pt2.Place.replace(
        merge_residence_values,"RESIDENCE")

In [12]:
merge_school_values = ['SCHOOL - PRIVATE BUILDING',
                       'SCHOOL - PRIVATE GROUNDS',
                       'SCHOOL - PUBLIC BUILDING',
                       'SCHOOL - PUBLIC GROUNDS',
                       'SCHOOL, PRIVATE, BUILDING',
                       'SCHOOL, PRIVATE, GROUNDS',
                       'SCHOOL, PUBLIC, BUILDING',
                       'SCHOOL, PUBLIC, GROUNDS',
                       'COLLEGE / UNIVERSITY - GROUNDS',
                       'COLLEGE/UNIVERSITY GROUNDS',
                       'COLLEGE/UNIVERSITY RESIDENCE HALL']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Place = cleaned_df_pt2.Place.replace(
        merge_school_values,"SCHOOL/COLLEGE")

In [13]:
merge_parking_lot_values = ['PARKING LOT',
                            'PARKING LOT / GARAGE (NON RESIDENTIAL)',
                            'PARKING LOT/GARAGE(NON.RESID.)']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Place = cleaned_df_pt2.Place.replace(
        merge_parking_lot_values,"PARKING LOT")

In [14]:
merge_public_transit_values = ['CTA BUS',
                               'CTA BUS STOP',
                               'CTA GARAGE / OTHER PROPERTY',
                               'CTA PARKING LOT / GARAGE / OTHER PROPERTY',
                               'CTA PLATFORM',
                               'CTA STATION',
                               'CTA TRAIN',
                               'OTHER COMMERCIAL TRANSPORTATION',
                               'OTHER RAILROAD PROP / TRAIN DEPOT',
                               'TAXICAB',
                               'PUBLIC TRANSIT',
                               'VEHICLE - OTHER RIDE SERVICE',
                               'VEHICLE - OTHER RIDE SHARE SERVICE (E.G., UBER, LYFT)',
                               'VEHICLE - OTHER RIDE SHARE SERVICE (LYFT, UBER, ETC.)']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Place = cleaned_df_pt2.Place.replace(
        merge_public_transit_values,"PUBLIC TRANSIT")

In [15]:
merge_cha_values = ['CHA APARTMENT',
                    'CHA HALLWAY / STAIRWELL / ELEVATOR',
                    'CHA HALLWAY/STAIRWELL/ELEVATOR',
                    'CHA PARKING LOT / GROUNDS',
                    'CHA PARKING LOT/GROUNDS']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Place = cleaned_df_pt2.Place.replace(
        merge_cha_values,"CHICAGO HOUSING AUTH")

In [16]:
merge_vehicle_values = ['VEHICLE NON-COMMERCIAL',
                        'AUTO']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Place = cleaned_df_pt2.Place.replace(
        merge_vehicle_values,"VEHICLE PERSONAL")

In [17]:
merge_gasconv_values = ['CONVENIENCE STORE',
                        'GAS STATION']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Place = cleaned_df_pt2.Place.replace(
        merge_gasconv_values,"GAS STATION CONVENIENCE")

In [18]:
merge_gasconv_values = ['SMALL RETAIL STORE',
                        'RETAIL STORE',
                        'DEPARTMENT STORE']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Place = cleaned_df_pt2.Place.replace(
        merge_gasconv_values,"GAS STATION CONVENIENCE")

In [19]:
# Check to make sure binning was successful
Place_counts = cleaned_df_pt2.Place.value_counts()
pd.options.display.max_rows = 4000
Place_counts

RESIDENCE                                          19487
STREET                                             11868
SIDEWALK                                            3266
GAS STATION CONVENIENCE                             3175
PARKING LOT                                         1487
ALLEY                                               1095
PUBLIC TRANSIT                                      1079
RESTAURANT                                          1052
OTHER                                                798
VEHICLE PERSONAL                                     758
SCHOOL/COLLEGE                                       648
OTHER (SPECIFY)                                      585
GROCERY FOOD STORE                                   583
COMMERCIAL / BUSINESS OFFICE                         468
BAR OR TAVERN                                        321
PARK PROPERTY                                        295
CHICAGO HOUSING AUTH                                 268
DRUG STORE                     

In [20]:
# Determine which values to replace if counts are less than ...?
replace_Place = list(Place_counts[Place_counts < 800].index)

# Replace in dataframe
for Place in replace_Place:
    cleaned_df_pt2.Place = cleaned_df_pt2.Place.replace(Place,"Other")
    
# Check to make sure binning was successful
cleaned_df_pt2.Place.value_counts()

RESIDENCE                  19487
STREET                     11868
Other                       7491
SIDEWALK                    3266
GAS STATION CONVENIENCE     3175
PARKING LOT                 1487
ALLEY                       1095
PUBLIC TRANSIT              1079
RESTAURANT                  1052
Name: Place, dtype: int64

## Create new columns & bins for modeling "When" the crime happened
- DateTime_Numeric
- Date
- Day_of_Week
- Time
- Time_of_Day
    - Grouped by: 
        - EarlyMorning 2am-6am
        - Morning 6am-10am
        - Midday 10am-2pm
        - Afternoon 2pm-6pm
        - Evening 6pm-10pm
        - Night 10pm-2am
        
See: [Pandas Datetime](https://pandas.pydata.org/docs/getting_started/intro_tutorials/09_timeseries.html) | [Datetime components](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-components) | [Binning the 'Time_of_Day'](https://stackoverflow.com/questions/33151463/how-to-bin-time-in-a-pandas-dataframe)


In [21]:
# What is the DateTime data type currently?
cleaned_df_pt2.DateTime.head(1)

0    02/02/2021 12:57:00 AM
Name: DateTime, dtype: object

In [22]:
# Convert to DateTime column's object datatype to panda's datetime data type
cleaned_df_pt2['DateTime'] = pd.to_datetime(cleaned_df_pt2['DateTime'])
cleaned_df_pt2.DateTime.head(1)

0   2021-02-02 00:57:00
Name: DateTime, dtype: datetime64[ns]

In [23]:
# Create 'DateTime_Numeric' column & convert datatype to integer
cleaned_df_pt2['DateTime_Numeric'] =pd.to_numeric(cleaned_df_pt2['DateTime'])
cleaned_df_pt2.DateTime_Numeric.head()

0    1612227420000000000
1    1568287800000000000
2    1639627080000000000
3    1636891200000000000
4    1637727300000000000
Name: DateTime_Numeric, dtype: int64

In [24]:
# Create 'Date' column
cleaned_df_pt2["Date"] = cleaned_df_pt2["DateTime"].dt.date
cleaned_df_pt2.Date.head()

0    2021-02-02
1    2019-09-12
2    2021-12-16
3    2021-11-14
4    2021-11-24
Name: Date, dtype: object

In [25]:
# Create 'Day_Num' column  (Monday=0, Sunday=6)
cleaned_df_pt2["Day_Num"] = cleaned_df_pt2["DateTime"].dt.day_of_week
cleaned_df_pt2.Day_Num.head()

0    1
1    3
2    3
3    6
4    2
Name: Day_Num, dtype: int64

In [26]:
# Create 'Day_SMTWTFS' column  
cleaned_df_pt2["Day_SMTWTFS"] = cleaned_df_pt2["DateTime"].dt.day_of_week
cleaned_df_pt2.Day_SMTWTFS.head()

0    1
1    3
2    3
3    6
4    2
Name: Day_SMTWTFS, dtype: int64

In [27]:
# Change 'Day_SMTWTFS' values to days of the week string data types  

cleaned_df_pt2['Day_SMTWTFS'] = cleaned_df_pt2['Day_SMTWTFS'].replace(
    [0,1,2,3,4,5,6],
    ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
)

# # cleaned_df_pt2["Day_SMTWTFS"] = cleaned_df_pt2["DateTime"].dt.day_name
cleaned_df_pt2.Day_SMTWTFS.head()

0      Tuesday
1     Thursday
2     Thursday
3       Sunday
4    Wednesday
Name: Day_SMTWTFS, dtype: object

In [28]:
# Create 'Time' column  
cleaned_df_pt2["Time"] = cleaned_df_pt2["DateTime"].dt.time
cleaned_df_pt2.Time.head()

0    00:57:00
1    11:30:00
2    03:58:00
3    12:00:00
4    04:15:00
Name: Time, dtype: object

In [29]:
# Create 'Time_of_Day' column and bin the vaules based of the time of hte day as defined by:
    # Night        12am-2am  00:00:00 - 01:59:59   
    # EarlyMorning 2am-6am   02:00:00 - 05:59:59   
    # Morning      6am-10am  06:00:00 - 09:59:59   
    # Midday      10am-2pm   10:00:00 - 13:59:59
    # Afternoon   2pm-6pm   14:00:00 - 17:59:59
    # Evening     6pm-10pm  18:00:00 - 21:59:59
    # Night       10pm-12am  22:00:00 - 23:59:59

# Define the bins (number represents the begining of the range, in relation to the ordered list of 'labels' below)
bins = [0, 2, 6, 10, 14, 18, 22, 24]

# Add labels fo rthe bins
labels = ['Night', 'Early Morning', 'Morning', 'Midday', 'Afternoon', 'Evening', 'Night']

# Add Time_of_Day column for the binned data to the dataframe
cleaned_df_pt2['Time_of_Day'] = pd.cut(cleaned_df_pt2.DateTime.dt.hour, bins, labels=labels, right=False, ordered=False)

cleaned_df_pt2.Time_of_Day.head()

0            Night
1           Midday
2    Early Morning
3           Midday
4    Early Morning
Name: Time_of_Day, dtype: category
Categories (6, object): ['Afternoon', 'Early Morning', 'Evening', 'Midday', 'Morning', 'Night']

In [30]:
# View the cleaned data pt.2 df so far
cleaned_df_pt2.head()

,ID,DateTime,Offense,Offense_Description,Place,Arrest,Domestic,Police_Dist,Community_No,Latitude,Longitude,LatLong,DateTime_Numeric,Date,Day_Num,Day_SMTWTFS,Time,Time_of_Day
0,12283861,2021-02-02 00:57:00,NON-VIOLENT OTHER,UNLAWFUL POSSESSION - HANDGUN,STREET,True,False,3.0,69.0,41.763110,-87.615232,"(41.763110324, -87.615231799)",1612227420000000000,2021-02-02,1,Tuesday,00:57:00,Night
1,11825618,2019-09-12 11:30:00,THEFT,RETAIL THEFT,Other,False,False,12.0,28.0,41.866989,-87.657205,"(41.866988841, -87.657205006)",1568287800000000000,2019-09-12,3,Thursday,11:30:00,Midday
2,12568929,2021-12-16 03:58:00,CRIMINAL DAMAGE,TO PROPERTY,RESTAURANT,False,False,6.0,44.0,41.736520,-87.613403,"(41.736520302, -87.613403026)",1639627080000000000,2021-12-16,3,Thursday,03:58:00,Early Morning
3,12542403,2021-11-14 12:00:00,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,12.0,24.0,41.901872,-87.699330,"(41.901872469, -87.699330295)",1636891200000000000,2021-11-14,6,Sunday,12:00:00,Midday
4,12552301,2021-11-24 04:15:00,BATTERY,SIMPLE,PUBLIC TRANSIT,False,False,16.0,76.0,41.978108,-87.904123,"(41.978108446, -87.904122758)",1637727300000000000,2021-11-24,2,Wednesday,04:15:00,Early Morning


## The 'Community' column: 

Indicates the community area where the incident occurred. Chicago has 77 community areas.  [Community areas](https://data.cityofchicago.org/d/cauq-8yn6)| [Community Areas Names](https://en.wikipedia.org/wiki/Community_areas_in_Chicago)
1. Create a 'Community_Names' column to more easily use call the community by name for visualizations
2. 77 is a large number of groups for modeling, so we'll bin Communities into 9 [Regions, as defined by Peter Fitzgerald](https://en.wikipedia.org/wiki/Community_areas_in_Chicago#/media/File:Chicago_community_areas_map.svg) called 'Community_Group'.

### Add Community Name

In [31]:
# Read in a dataset containing the the community names
url = "https://datahub.cmap.illinois.gov/dataset/1d2dd970-f0a6-4736-96a1-3caeb431f5e4/resource/8c4e096e-c90c-4bef-9cf1-9028d094296e/download/ReferenceCCAProfiles20152019.csv" 
keep = ['GEOID', 'GEOG']

community_areas_df = pd.read_csv(url, usecols = keep)
community_areas_df.head()

,GEOID,GEOG
0,14.0,Albany Park
1,57.0,Archer Heights
2,34.0,Armour Square
3,70.0,Ashburn
4,71.0,Auburn Gresham


In [32]:
# Rename columns
community_areas_df.rename(columns = {'GEOID':'Community_No', 'GEOG':'Community_Name'}, inplace = True)
community_areas_df.head()

,Community_No,Community_Name
0,14.0,Albany Park
1,57.0,Archer Heights
2,34.0,Armour Square
3,70.0,Ashburn
4,71.0,Auburn Gresham


In [33]:
# Merge the data on Community_No to add the proper Community_Name

cleaned_df_pt2 = pd.merge(cleaned_df_pt2, community_areas_df[['Community_No', 'Community_Name']], on='Community_No')


cleaned_df_pt2.sample(5)

,ID,DateTime,Offense,Offense_Description,Place,Arrest,Domestic,Police_Dist,Community_No,Latitude,Longitude,LatLong,DateTime_Numeric,Date,Day_Num,Day_SMTWTFS,Time,Time_of_Day,Community_Name
14706,12602121,2022-01-25 00:00:00,THEFT,OVER $500,STREET,False,False,2.0,38.0,41.805772,-87.614675,"(41.805772273, -87.614674898)",1643068800000000000,2022-01-25,1,Tuesday,00:00:00,Night,Grand Boulevard
27472,11775016,2019-07-29 00:45:00,BATTERY,AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON,RESIDENCE,False,True,5.0,49.0,41.689856,-87.624043,"(41.689856173, -87.624043246)",1564361100000000000,2019-07-29,0,Monday,00:45:00,Night,Roseland
30185,12162635,2020-09-11 12:27:00,BATTERY,DOMESTIC BATTERY SIMPLE,PARKING LOT,False,True,8.0,65.0,41.754593,-87.741529,"(41.754592961, -87.741528537)",1599827220000000000,2020-09-11,4,Friday,12:27:00,Midday,West Lawn
18974,12507342,2021-10-09 12:30:00,CRIMINAL DAMAGE,TO PROPERTY,Other,False,False,20.0,2.0,41.987451,-87.689569,"(41.987451426, -87.689569439)",1633782600000000000,2021-10-09,5,Saturday,12:30:00,Midday,West Ridge
34502,11844999,2019-09-22 21:00:00,SEXUAL CRIME,NON-AGGRAVATED,ALLEY,False,False,19.0,6.0,41.954479,-87.655152,"(41.954479259, -87.655152427)",1569186000000000000,2019-09-22,6,Sunday,21:00:00,Evening,Lake View


### Bin the Communities in new column: Community_Group
<image src="Images/Chicago_Grouped_Communities_Map.PNG" width="400" height="500">

In [34]:
# Check the number of unique values in Community_Name

Place_counts = cleaned_df_pt2.Community_Name.value_counts()
pd.options.display.max_rows = 4000
Place_counts

Austin                    2886
Near North Side           2120
South Shore               1804
Near West Side            1650
North Lawndale            1644
The Loop                  1556
Humboldt Park             1478
Auburn Gresham            1454
Roseland                  1365
Greater Grand Crossing    1360
West Englewood            1330
Chatham                   1290
West Town                 1278
West Garfield Park        1165
Englewood                 1132
Lake View                 1124
Chicago Lawn              1111
New City                   889
Logan Square               884
South Chicago              882
East Garfield Park         881
Belmont Cragin             856
West Pullman               849
South Lawndale             827
Rogers Park                776
Lincoln Park               731
Uptown                     702
West Ridge                 701
Woodlawn                   667
Grand Boulevard            664
Portage Park               620
Washington Heights         592
Irving P

In [35]:
# Create 'Community_Group' column  
cleaned_df_pt2["Community_Group"] = cleaned_df_pt2["Community_Name"]
cleaned_df_pt2.head(1)

,ID,DateTime,Offense,Offense_Description,Place,Arrest,Domestic,Police_Dist,Community_No,Latitude,Longitude,LatLong,DateTime_Numeric,Date,Day_Num,Day_SMTWTFS,Time,Time_of_Day,Community_Name,Community_Group
0,12283861,2021-02-02 00:57:00,NON-VIOLENT OTHER,UNLAWFUL POSSESSION - HANDGUN,STREET,True,False,3.0,69.0,41.76311,-87.615232,"(41.763110324, -87.615231799)",1612227420000000000,2021-02-02,1,Tuesday,00:57:00,Night,Greater Grand Crossing,Greater Grand Crossing


In [36]:
# Create the bins for each Community Group

merge_Far_North_values = ['Rogers Park',
                          'West Ridge',
                          'Uptown',
                          'Lincoln Square',
                          'Edison Park',
                          'Norwood Park',
                          'Jefferson Park',
                          'Forest Glen',
                          'North Park',
                          'Albany Park',
                          "O'Hare",
                          'Edgewater']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Community_Group = cleaned_df_pt2.Community_Group.replace(
        merge_Far_North_values,"Far North")

In [37]:
merge_Northwest_values = ['Portage Park',
                          'Irving Park',
                          'Dunning',
                          'Montclare',
                          'Belmont Cragin',
                          'Hermosa']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Community_Group = cleaned_df_pt2.Community_Group.replace(
        merge_Northwest_values,"Northwest")

In [38]:
merge_North_values = ['North Center',
                      'Lake View',
                      'Lincoln Park',
                      'Avondale',
                      'Logan Square']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Community_Group = cleaned_df_pt2.Community_Group.replace(
        merge_North_values,"North")

In [39]:
merge_Central_values = ['Near North Side',
                        'The Loop',
                        'Near South Side']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Community_Group = cleaned_df_pt2.Community_Group.replace(
        merge_Central_values,"Central")

In [40]:
merge_West_values = ['Humboldt Park',
                     'West Town',
                     'Austin',
                     'West Garfield Park',
                     'East Garfield Park',
                     'Near West Side',
                     'North Lawndale',
                     'South Lawndale',
                     'Lower West Side']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Community_Group = cleaned_df_pt2.Community_Group.replace(
        merge_West_values,"West")

In [41]:
merge_South_values = ['Armour Square',
                      'Douglas',
                      'Oakland',
                      'Fuller Park',
                      'Grand Boulevard',
                      'Kenwood',
                      'Washington Park',
                      'Hyde Park',
                      'Woodlawn',
                      'South Shore',
                      'Bridgeport',
                      'Greater Grand Crossing']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Community_Group = cleaned_df_pt2.Community_Group.replace(
        merge_South_values,"South")

In [42]:
merge_Southwest_values = ['Garfield Ridge',
                          'Archer Heights',
                          'Brighton Park',
                          'McKinley Park',
                          'New City',
                          'West Elsdon',
                          'Gage Park',
                          'Clearing',
                          'West Lawn',
                          'Chicago Lawn',
                          'West Englewood',
                          'Englewood']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Community_Group = cleaned_df_pt2.Community_Group.replace(
        merge_Southwest_values,"Southwest")

In [43]:
merge_Far_Southwest_values = ['Ashburn',
                              'Auburn Gresham',
                              'Beverly',
                              'Washington Heights',
                              'Mount Greenwood',
                              'Morgan Park']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Community_Group = cleaned_df_pt2.Community_Group.replace(
        merge_Far_Southwest_values,"Far Soutwest")

In [44]:
merge_Southeast_values = ['Chatham',
                          'Avalon Park',
                          'South Chicago',
                          'Burnside',
                          'Calumet Heights',
                          'Roseland',
                          'Pullman',
                          'South Deering',
                          'East Side',
                          'West Pullman',
                          'Riverdale',
                          'Hegewisch']

# Replace in DataFrame
for Place in cleaned_df_pt2:
    cleaned_df_pt2.Community_Group = cleaned_df_pt2.Community_Group.replace(
        merge_Southeast_values,"Southeast")

In [45]:
# Check to make sure binning was successful
cleaned_df_pt2.Community_Group.value_counts()

West            12321
South            6932
Southwest        6647
Southeast        6200
Far North        4603
Central          4048
North            3399
Far Soutwest     3154
Northwest        2696
Name: Community_Group, dtype: int64

## Bin 'Police_Dist' column?  
[District info](https://data.cityofchicago.org/d/fthy-xz3r)

In [46]:
# Check the number of unique values in 'District' column
print(cleaned_df_pt2.Police_Dist.nunique())

23


Although there are 23, which is more than the goal for 10 or less ideal for modeling and visualizations, let's not bin these because police districts are the largest region of division for police in Chicago. [See all the districts here](https://home.chicagopolice.org/about/police-districts/1st-district-central/)

In [47]:
cleaned_df_pt2.head()

,ID,DateTime,Offense,Offense_Description,Place,Arrest,Domestic,Police_Dist,Community_No,Latitude,Longitude,LatLong,DateTime_Numeric,Date,Day_Num,Day_SMTWTFS,Time,Time_of_Day,Community_Name,Community_Group
0,12283861,2021-02-02 00:57:00,NON-VIOLENT OTHER,UNLAWFUL POSSESSION - HANDGUN,STREET,True,False,3.0,69.0,41.763110,-87.615232,"(41.763110324, -87.615231799)",1612227420000000000,2021-02-02,1,Tuesday,00:57:00,Night,Greater Grand Crossing,South
1,11767977,2019-07-21 17:00:00,THEFT,$500 AND UNDER,Other,False,False,3.0,69.0,41.762927,-87.595774,"(41.762926686, -87.595774261)",1563728400000000000,2019-07-21,6,Sunday,17:00:00,Afternoon,Greater Grand Crossing,South
2,12308286,2021-03-06 02:45:00,ASSAULT,SIMPLE,RESIDENCE,False,False,3.0,69.0,41.767020,-87.595181,"(41.767020109, -87.595181198)",1614998700000000000,2021-03-06,5,Saturday,02:45:00,Early Morning,Greater Grand Crossing,South
3,11681265,2019-05-08 20:00:00,BATTERY,SIMPLE,RESIDENCE,False,False,7.0,69.0,41.770208,-87.628291,"(41.770208372, -87.628290908)",1557345600000000000,2019-05-08,2,Wednesday,20:00:00,Evening,Greater Grand Crossing,South
4,11831071,2019-09-13 07:30:00,THEFT,$500 AND UNDER,STREET,False,False,7.0,69.0,41.764603,-87.633422,"(41.764602959, -87.633421657)",1568359800000000000,2019-09-13,4,Friday,07:30:00,Morning,Greater Grand Crossing,South


In [48]:
#Reorder the Column Names
column_names = ['ID', 
                'Offense', 
                'Offense_Description', 
                'Arrest', 
                'Domestic',
                'DateTime',
                'DateTime_Numeric',
                'Date',
                'Day_Num',
                'Day_SMTWTFS',
                'Time',
                'Time_of_Day',
                'Place',
                'Police_Dist',
                'Community_No',
                'Community_Name',
                'Community_Group',
                'Latitude',
                'Longitude',
                'LatLong']
cleaned_df_pt2 = cleaned_df_pt2.reindex(columns=column_names)

cleaned_df_pt2.head()

,ID,Offense,Offense_Description,Arrest,Domestic,DateTime,DateTime_Numeric,Date,Day_Num,Day_SMTWTFS,Time,Time_of_Day,Place,Police_Dist,Community_No,Community_Name,Community_Group,Latitude,Longitude,LatLong
0,12283861,NON-VIOLENT OTHER,UNLAWFUL POSSESSION - HANDGUN,True,False,2021-02-02 00:57:00,1612227420000000000,2021-02-02,1,Tuesday,00:57:00,Night,STREET,3.0,69.0,Greater Grand Crossing,South,41.763110,-87.615232,"(41.763110324, -87.615231799)"
1,11767977,THEFT,$500 AND UNDER,False,False,2019-07-21 17:00:00,1563728400000000000,2019-07-21,6,Sunday,17:00:00,Afternoon,Other,3.0,69.0,Greater Grand Crossing,South,41.762927,-87.595774,"(41.762926686, -87.595774261)"
2,12308286,ASSAULT,SIMPLE,False,False,2021-03-06 02:45:00,1614998700000000000,2021-03-06,5,Saturday,02:45:00,Early Morning,RESIDENCE,3.0,69.0,Greater Grand Crossing,South,41.767020,-87.595181,"(41.767020109, -87.595181198)"
3,11681265,BATTERY,SIMPLE,False,False,2019-05-08 20:00:00,1557345600000000000,2019-05-08,2,Wednesday,20:00:00,Evening,RESIDENCE,7.0,69.0,Greater Grand Crossing,South,41.770208,-87.628291,"(41.770208372, -87.628290908)"
4,11831071,THEFT,$500 AND UNDER,False,False,2019-09-13 07:30:00,1568359800000000000,2019-09-13,4,Friday,07:30:00,Morning,STREET,7.0,69.0,Greater Grand Crossing,South,41.764603,-87.633422,"(41.764602959, -87.633421657)"


In [49]:
cleaned_df_pt2.dtypes

ID                              int64
Offense                        object
Offense_Description            object
Arrest                           bool
Domestic                         bool
DateTime               datetime64[ns]
DateTime_Numeric                int64
Date                           object
Day_Num                         int64
Day_SMTWTFS                    object
Time                           object
Time_of_Day                  category
Place                          object
Police_Dist                   float64
Community_No                  float64
Community_Name                 object
Community_Group                object
Latitude                      float64
Longitude                     float64
LatLong                        object
dtype: object

In [50]:
# Saving cleaned data
cleaned_df_pt2.to_csv("Resources/Chicago_Crimes_cleaned_pt2.csv", index=False)